In [0]:
%python
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "69540130-9121-4441-93db-f1dc4498baa3",
    "fs.azure.account.oauth2.client.secret": '',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/f8ef5927-40a6-4fb5-a1ef-81ff563ee62d/oauth2/token"
}

mount_point = "/mnt/chargesafe"

# Check if the directory is already mounted
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

dbutils.fs.mount(
    source="abfss://credit-card-data@chargesafedata.dfs.core.windows.net",  # container@storageacc
    mount_point=mount_point,
    extra_configs=configs
)

/mnt/chargesafe has been unmounted.
Out[3]: True

In [0]:
%fs
ls "/mnt/chargesafe"

path,name,size,modificationTime
dbfs:/mnt/chargesafe/raw-data/,raw-data/,0,1734765234000
dbfs:/mnt/chargesafe/transformed-data/,transformed-data/,0,1734765250000


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
data = spark.read.format("csv").option("header", "true").load("/mnt/chargesafe/raw-data/data.csv")

In [0]:
data.show(5)

+-------+---------------------+-----------+--------------------+--------------+-----+------+--------+------+--------------------+----------+-----+-----+-------+--------+--------+--------------------+----------+--------------------+----------+---------+-----------+--------+
|sr. no.|trans_date_trans_time|     cc_num|            merchant|      category|  amt| first|    last|gender|              street|      city|state|  zip|    lat|    long|city_pop|                 job|       dob|           trans_num| unix_time|merch_lat| merch_long|is_fraud|
+-------+---------------------+-----------+--------------------+--------------+-----+------+--------+------+--------------------+----------+-----+-----+-------+--------+--------+--------------------+----------+--------------------+----------+---------+-----------+--------+
|      1|     21-06-2020 12:14|2.29116E+15|fraud_Kirlin and ...| personal_care| 2.86|  Jeff| Elliott|     M|   351 Darlene Green|  Columbia|   SC|29209|33.9659|-80.9355|  333497|

In [0]:
data.printSchema()

root
 |-- sr. no.: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city_pop: string (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- is_fraud: string (nullable = true)



In [0]:
columns_to_drop = ["unix_time", "merch_lat", "merch_long", "city_pop", "zip", "lat", "long", "job", "sr. no.", "street", "cc_num"]
data = data.drop(*columns_to_drop)

In [0]:
data.show(5)

+---------------------+--------------------+--------------+-----+------+--------+------+----------+-----+----------+--------------------+--------+
|trans_date_trans_time|            merchant|      category|  amt| first|    last|gender|      city|state|       dob|           trans_num|is_fraud|
+---------------------+--------------------+--------------+-----+------+--------+------+----------+-----+----------+--------------------+--------+
|     21-06-2020 12:14|fraud_Kirlin and ...| personal_care| 2.86|  Jeff| Elliott|     M|  Columbia|   SC|19-03-1968|2da90c7d74bd46a0c...|       0|
|     21-06-2020 12:14|fraud_Sporer-Keebler| personal_care|29.84|Joanne|Williams|     F|   Altonah|   UT|17-01-1990|324cc204407e99f51...|       0|
|     21-06-2020 12:14|fraud_Swaniawski,...|health_fitness|41.28|Ashley|   Lopez|     F|  Bellmore|   NY|21-10-1970|c81755dbbbea9d5c7...|       0|
|     21-06-2020 12:15|   fraud_Haley Group|      misc_pos|60.05| Brian|Williams|     M|Titusville|   FL|25-07-1987|21

In [0]:
data.printSchema()

root
 |-- trans_date_trans_time: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)



In [0]:
%python
from pyspark.sql.functions import concat, lit

data_with_name = data.withColumn("customer", concat(data["first"], lit(" "), data["last"]))
data = data_with_name.drop("first", "last")
data.show(5)

+---------------------+--------------------+--------------+-----+------+----------+-----+----------+--------------------+--------+---------------+
|trans_date_trans_time|            merchant|      category|  amt|gender|      city|state|       dob|           trans_num|is_fraud|       customer|
+---------------------+--------------------+--------------+-----+------+----------+-----+----------+--------------------+--------+---------------+
|     21-06-2020 12:14|fraud_Kirlin and ...| personal_care| 2.86|     M|  Columbia|   SC|19-03-1968|2da90c7d74bd46a0c...|       0|   Jeff Elliott|
|     21-06-2020 12:14|fraud_Sporer-Keebler| personal_care|29.84|     F|   Altonah|   UT|17-01-1990|324cc204407e99f51...|       0|Joanne Williams|
|     21-06-2020 12:14|fraud_Swaniawski,...|health_fitness|41.28|     F|  Bellmore|   NY|21-10-1970|c81755dbbbea9d5c7...|       0|   Ashley Lopez|
|     21-06-2020 12:15|   fraud_Haley Group|      misc_pos|60.05|     M|Titusville|   FL|25-07-1987|2159175b9efe66dc3.

In [0]:
data.printSchema()

root
 |-- trans_date_trans_time: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- customer: string (nullable = true)



In [0]:
data.select("trans_date_trans_time").show(truncate=False, n=5)

+---------------------+
|trans_date_trans_time|
+---------------------+
|21-06-2020 12:14     |
|21-06-2020 12:14     |
|21-06-2020 12:14     |
|21-06-2020 12:15     |
|21-06-2020 12:15     |
+---------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import to_timestamp, col
data = data.withColumn("trans_date_trans_time", to_timestamp(col("trans_date_trans_time"), "dd-MM-yyyy HH:mm"))

In [0]:
from pyspark.sql.types import DoubleType, TimestampType, IntegerType, DateType, StringType
datatype_mapping = {
    "trans_date_trans_time": TimestampType(),
    "amt": DoubleType(),
    "is_fraud": IntegerType()
}

for column, new_type in datatype_mapping.items():
    data = data.withColumn(column, data[column].cast(new_type))

In [0]:
from pyspark.sql.functions import substring; 
data = data.withColumn("dob", substring("dob", -4, 4))
data = data.withColumn("dob", data["dob"].cast(IntegerType()))

In [0]:
from pyspark.sql.functions import regexp_replace
data = data.withColumn("merchant", regexp_replace("merchant", "^fraud_", ""))

In [0]:
data = data.withColumnRenamed("amt", "amount in $")

In [0]:
new_column_order = ["trans_num", "trans_date_trans_time", "amount in $", "category", "merchant", "customer", "gender", "city", "state", "dob", "is_fraud"]
data = data.select(new_column_order)

In [0]:
data.show(5)

+--------------------+---------------------+-----------+--------------+--------------------+---------------+------+----------+-----+----+--------+
|           trans_num|trans_date_trans_time|amount in $|      category|            merchant|       customer|gender|      city|state| dob|is_fraud|
+--------------------+---------------------+-----------+--------------+--------------------+---------------+------+----------+-----+----+--------+
|2da90c7d74bd46a0c...|  2020-06-21 12:14:00|       2.86| personal_care|     Kirlin and Sons|   Jeff Elliott|     M|  Columbia|   SC|1968|       0|
|324cc204407e99f51...|  2020-06-21 12:14:00|      29.84| personal_care|      Sporer-Keebler|Joanne Williams|     F|   Altonah|   UT|1990|       0|
|c81755dbbbea9d5c7...|  2020-06-21 12:14:00|      41.28|health_fitness|Swaniawski, Nitzs...|   Ashley Lopez|     F|  Bellmore|   NY|1970|       0|
|2159175b9efe66dc3...|  2020-06-21 12:15:00|      60.05|      misc_pos|         Haley Group| Brian Williams|     M|Tit

In [0]:
data.printSchema()

root
 |-- trans_num: string (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- amount in $: double (nullable = true)
 |-- category: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- dob: integer (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [0]:
data.write.option("header", "true").csv("/mnt/chargesafe/transformed-data/data")